LANGKAH 1 - Import Library

In [1]:
pip install tensorflow

  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.7.1
    Uninstalling typing_extensions-4.7.1:
      Successfully uninstalled typing_extensions-4.7.1
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
python-lsp-black 1.2.1 requires black>=22.3.0, but you have black 0.0 which is incompatible.


In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

LANGKAH 2 - Load Data

In [3]:
dataset = pd.read_csv('Churn_Modelling.csv')
# Fitur X: semua kolom kecuali kolom pertama dan kolom terakhir.
X = dataset.iloc[:, 3:-1].values
# Target y: kolom terakhir dalam dataframe
y = dataset.iloc[:, -1].values

In [4]:
# cek data
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


LANGKAH 3 - Encoding Data Kategorikal

In [5]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
# konversi data kategorikal menjadi numerik
X[:, 2] = le.fit_transform(X[:, 2])

LANGKAH 4 - Encoding Kolom "Geography" dengan One Hot Encoder

One-hot encoding adalah teknik pengkodean data kategorikal yang mengubah setiap kategori menjadi vektor biner

In [6]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
# melakukan one hot encoding pada 1 kolom
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
# melakukan one-hot encoding pada kolom 1 dari dataframe X menggunakan objek ct
# Hasilnya akan disimpan dalam variabel X sebagai array NumPy.
X = np.array(ct.fit_transform(X))

LANGKAH 5 - Split Data

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

LANGKAH 6 - Scaling Fitur

In [8]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
# melakukan standarisasi pada data latih X_train menggunakan objek sc
X_train = sc.fit_transform(X_train)
# melakukan standarisasi pada data uji X_test menggunakan objek sc
X_test = sc.transform(X_test)

MEMBUAT MODEL ANN

LANGKAH 1 - Inisiasi Model ANN

In [9]:
# membuat model ANN baru menggunakan kelas Sequential
ann = tf.keras.models.Sequential()

LANGKAH 2 - Membuat Input Layer dan Hidden Layer Pertama

In [10]:
# Lapisan ini memiliki 6 neuron dan menggunakan fungsi aktivasi ReLU
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

LANGKAH 3 - Membuat Hidden Layer Kedua

In [11]:
# Lapisan ini memiliki 6 neuron dan menggunakan fungsi aktivasi ReLU
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

LANGKAH 4 - Membuat Output Layer

In [12]:
# Lapisan ini memiliki 1 neuron dan menggunakan fungsi aktivasi sigmoid
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

TRAINING MODEL

LANGKAH 1 - Compile Model (Menyatukan Arsitektur) ANN

In [13]:
#  mengkompilasi model ANN dengan menggunakan optimizer Adam, fungsi loss binary crossentropy, dan metrik akurasi
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

LANGKAH 2 - Fitting Model

In [14]:
# melatih model ANN menggunakan data latih X_train dan target latih y_train dengan ukuran batch 32 dan selama 100 epoch
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 13s 14ms/step - loss: 0.5568 - accuracy: 0.7448
Epoch 2/100
250/250 [==============================] - 2s 8ms/step - loss: 0.4678 - accuracy: 0.8011
Epoch 3/100
250/250 [==============================] - 1s 6ms/step - loss: 0.4396 - accuracy: 0.8111
Epoch 4/100
250/250 [==============================] - 1s 5ms/step - loss: 0.4245 - accuracy: 0.8165
Epoch 5/100
250/250 [==============================] - 1s 5ms/step - loss: 0.4147 - accuracy: 0.8211
Epoch 6/100
250/250 [==============================] - 1s 6ms/step - loss: 0.4068 - accuracy: 0.8221
Epoch 7/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3983 - accuracy: 0.8246
Epoch 8/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3873 - accuracy: 0.8288
Epoch 9/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3771 - accuracy: 0.8390
Epoch 10/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3680 - accu

Modelkan Data Baru dan Buat Prediksi

In [15]:
# 
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

1/1 [==============================] - 1s 873ms/step
[[False]]


Prediksi Dengan Data Testing

In [16]:
# Hasil prediksi akan disimpan dalam variabel y_pred 
y_pred = ann.predict(X_test)
# hasil prediksi akan dikonversi menjadi nilai biner (0 atau 1) dengan menggunakan fungsi > 0.5
y_pred = (y_pred > 0.5)
# mencetak hasil gabungan dari hasil prediksi y_pred dan target uji y_test menjadi satu matriks
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

63/63 [==============================] - 1s 10ms/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


Cek Akurasi dan Confusion Matrix

In [17]:
from sklearn.metrics import confusion_matrix, accuracy_score
# menghitung confusion amtrix untuk data test dan hasil prediksi
cm = confusion_matrix(y_test, y_pred)
print(cm)
# menghitung akurasi model klasifikasi pada data test dan hasil prediksi
accuracy_score(y_test, y_pred)

[[1519   76]
 [ 195  210]]


0.8645